In [1]:
from google.colab import drive 

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!unzip "/content/gdrive/My Drive/Colab Notebooks/data.h5.zip" -d "./"

Archive:  /content/gdrive/My Drive/Colab Notebooks/data.h5.zip
  inflating: ./data1.h5              


In [3]:
"""
Author: Pranav Srivastava
file: PartB.py
This file has implementation of PartB (i) and (ii)

execution: functions with 10 variants of feature  extraction and 4 variants of fine tuning are available at the bottom of this file
"""

import numpy as np
import h5py
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from contextlib import redirect_stdout

NUM_EPOCHS = 100

def loadDataH5():
    with h5py.File('data1.h5','r') as hf:
        trainX = np.array(hf.get('trainX'))
        trainY = np.array(hf.get('trainY'))
        valX = np.array(hf.get('valX'))
        valY = np.array(hf.get('valY'))
        print (trainX.shape,trainY.shape)
        print (valX.shape,valY.shape)
    return trainX, trainY, valX, valY

trainX, trainY, testX, testY = loadDataH5()

def save_accuracy_score(filename, r, labels):
    path = "/content/gdrive/My Drive/Colab Notebooks/accuracy_results/"
    filename = path+filename+".txt"

    accuracy_score = metrics.accuracy_score(r, labels)

    with open(filename, 'w') as f:
        f.write(str(accuracy_score))
        f.close()

def save_model_summary(filename, model):
    path = "/content/gdrive/My Drive/Colab Notebooks/model_summary/"
    filename = path+filename+".txt"

    with open(filename, 'w') as f:
        with redirect_stdout(f):
            model.summary()
            f.close

def plotAccLoss(H, NUM_EPOCHS):

    plt.style.use("ggplot")
    plt.figure()
    plt.plot(np.arange(0, NUM_EPOCHS), H.history["loss"], label="train_loss")
    plt.plot(np.arange(0, NUM_EPOCHS), H.history["val_loss"], label="val_loss")
    plt.plot(np.arange(0, NUM_EPOCHS), H.history["accuracy"], label="train_acc")
    plt.plot(np.arange(0, NUM_EPOCHS), H.history["val_accuracy"], label="val_acc")
    plt.title("Training Loss and Accuracy")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend(['train_loss', 'val_loss', 'train_acc', 'val_acc'], loc='upper right')
    plt.show()

def featureExtractionTransferLearning_variant1():

    """
    This is the variant of feature extraction
    VGG16+LogisticRegression
    Implementaion for PartB (i)
    """
    vggModel = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

    print (vggModel.summary())
    save_model_summary("FE_TL_VGG16_LogisticRegression", vggModel)

    featuresTrain = vggModel.predict(trainX)
    featuresTrain = featuresTrain.reshape(featuresTrain.shape[0], -1)

    featuresVal = vggModel.predict(testX)
    featuresVal = featuresVal.reshape(featuresVal.shape[0], -1)


    model = LogisticRegression()
    model.fit(featuresTrain, trainY)

    results = model.predict(featuresVal)
    print("results are --->")
    print (metrics.accuracy_score(results, testY))
    save_accuracy_score("FE_TL_VGG16_LogisticRegression", results, testY)


def featureExtractionTransferLearning_variant2():
    """
    This is the variant of feature extraction
    VGG16+RandomForestClassifier_500
    Implementaion for PartB (i)
    """
    vggModel = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

    print (vggModel.summary())
    save_model_summary("FE_TL_VGG16_RandomForestClassifier_500", vggModel)

    featuresTrain = vggModel.predict(trainX)
    featuresTrain = featuresTrain.reshape(featuresTrain.shape[0], -1)

    featuresVal = vggModel.predict(testX)
    featuresVal = featuresVal.reshape(featuresVal.shape[0], -1)

    # 500 estimators (trees)
    model = RandomForestClassifier(n_estimators=500)

    model.fit(featuresTrain, trainY)
    results = model.predict(featuresVal)
    print("results are --->")
    print (metrics.accuracy_score(results, testY))
    save_accuracy_score("FE_TL_VGG16_RandomForestClassifier_200", results, testY)

def featureExtractionTransferLearning_variant3():
    """
    This is the variant of feature extraction
    InceptionV3+LogisticRegression
    Implementaion for PartB (i)
    """
    initialModel = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

    print (initialModel.summary())
    save_model_summary("FE_TL_Inception_v3_LogisticRegression", initialModel)

    featuresTrain = initialModel.predict(trainX)
    featuresTrain = featuresTrain.reshape(featuresTrain.shape[0], -1)

    featuresVal = initialModel.predict(testX)
    featuresVal = featuresVal.reshape(featuresVal.shape[0], -1)

    model = LogisticRegression()

    model.fit(featuresTrain, trainY)
    results = model.predict(featuresVal)
    print("accuracy: ")
    print (metrics.accuracy_score(results, testY))
    save_accuracy_score("FE_TL_Inception_v3_LogisticRegression", results, testY)


def featureExtractionTransferLearning_variant4():
    """
    This is the variant of feature extraction
    InceptionV3+RandomForestClassifier_200
    Implementaion for PartB (i)
    """
    initialModel = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

    print (initialModel.summary())
    save_model_summary("FE_TL_Inception_v3_RandomForestClassifier_200", initialModel)

    featuresTrain = initialModel.predict(trainX)
    featuresTrain = featuresTrain.reshape(featuresTrain.shape[0], -1)

    featuresVal = initialModel.predict(testX)
    featuresVal = featuresVal.reshape(featuresVal.shape[0], -1)

    # 500 estimators (trees)
    model = RandomForestClassifier(n_estimators=200)

    model.fit(featuresTrain, trainY)
    results = model.predict(featuresVal)
    print("accuracy: ")
    print (metrics.accuracy_score(results, testY))
    save_accuracy_score("FE_TL_Inception_v3_RandomForestClassifier_200", results, testY)


def featureExtractionTransferLearning_variant5():
    """
    This is the variant of feature extraction
    ResNet152V2+LogisticRegression
    Implementaion for PartB (i)
    """
    initialModel = tf.keras.applications.ResNet152V2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

    print (initialModel.summary())
    save_model_summary("FE_TL_ResNet152V2_LogisticRegression", initialModel)

    featuresTrain = initialModel.predict(trainX)
    featuresTrain = featuresTrain.reshape(featuresTrain.shape[0], -1)

    featuresVal = initialModel.predict(testX)
    featuresVal = featuresVal.reshape(featuresVal.shape[0], -1)

    model = LogisticRegression()

    model.fit(featuresTrain, trainY)
    results = model.predict(featuresVal)
    print("accuracy: ")
    print (metrics.accuracy_score(results, testY))
    save_accuracy_score("FE_TL_ResNet152V2_LogisticRegression", results, testY)


def featureExtractionTransferLearning_variant6():
    """
    This is the variant of feature extraction
    ResNet152V2+RandomForestClassifier_200
    Implementaion for PartB (i)
    """
    initialModel = tf.keras.applications.ResNet152V2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

    print (initialModel.summary())
    save_model_summary("FE_TL_ResNet152V2_RandomForestClassifier_200", initialModel)

    featuresTrain = initialModel.predict(trainX)
    featuresTrain = featuresTrain.reshape(featuresTrain.shape[0], -1)

    featuresVal = initialModel.predict(testX)
    featuresVal = featuresVal.reshape(featuresVal.shape[0], -1)

    # 500 estimators (trees)
    model = RandomForestClassifier(n_estimators=200)

    model.fit(featuresTrain, trainY)
    results = model.predict(featuresVal)
    print("accuracy: ")
    print (metrics.accuracy_score(results, testY))
    save_accuracy_score("FE_TL_ResNet152V2_RandomForestClassifier_200", results, testY)


def featureExtractionTransferLearning_variant7():
    """
    This is the variant of feature extraction
    VGG16(till block4_conv2)+LogisticRegression
    Implementaion for PartB (i)
    """

    """Create a New Model Using a Portion of an Original Model"""

    vggModel = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

    #cut the original network at block4_conv2
    portionOfVGG16= tf.keras.Model(inputs=vggModel.input, outputs=vggModel.get_layer('block4_conv2').output)

    print (portionOfVGG16.summary())
    save_model_summary("FE_TL_portionOfVGG16_LogisticRegression", portionOfVGG16)

    featuresTrain = portionOfVGG16.predict(trainX)
    featuresTrain = featuresTrain.reshape(featuresTrain.shape[0], -1)

    featuresVal = portionOfVGG16.predict(testX)
    featuresVal = featuresVal.reshape(featuresVal.shape[0], -1)


    model = LogisticRegression()
    model.fit(featuresTrain, trainY)

    results = model.predict(featuresVal)
    print("results are --->")
    print (metrics.accuracy_score(results, testY))
    save_accuracy_score("FE_TL_portionOfVGG16_LogisticRegression", results, testY)


def featureExtractionTransferLearning_variant8():
    """
    This is the variant of feature extraction
    InceptionV3(till conv2d_50)+LogisticRegression
    Implementaion for PartB (i)
    """

    """Create a New Model Using a Portion of an Original Model"""

    initialModel = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

    #cut the original network at conv2d_50
    portionOfInitialModel= tf.keras.Model(inputs=initialModel.input, outputs=initialModel.get_layer('conv2d_50').output)

    print (portionOfInitialModel.summary())
    save_model_summary("FE_TL_portionOfInceptionV3_LogisticRegression", portionOfInitialModel)

    featuresTrain = portionOfInitialModel.predict(trainX)
    featuresTrain = featuresTrain.reshape(featuresTrain.shape[0], -1)

    featuresVal = portionOfInitialModel.predict(testX)
    featuresVal = featuresVal.reshape(featuresVal.shape[0], -1)


    model = LogisticRegression()
    model.fit(featuresTrain, trainY)

    results = model.predict(featuresVal)
    print("results are --->")
    print (metrics.accuracy_score(results, testY))
    save_accuracy_score("FE_TL_portionOfInceptionV3_LogisticRegression", results, testY)

def featureExtractionTransferLearning_variant9():

    """
    This is the variant of feature extraction
    ResNet152V2(till conv4_block23_2_conv)+LogisticRegression
    Implementaion for PartB (i)
    """

    """Create a New Model Using a Portion of an Original Model"""

    initialModel = tf.keras.applications.ResNet152V2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

    #cut the original network at conv4_block23_2_conv
    portionOfInitialModel= tf.keras.Model(inputs=initialModel.input, outputs=initialModel.get_layer('conv4_block23_2_conv').output)

    print (portionOfInitialModel.summary())
    save_model_summary("FE_TL_portionOfResNet152V2_LogisticRegression", portionOfInitialModel)

    featuresTrain = portionOfInitialModel.predict(trainX)
    featuresTrain = featuresTrain.reshape(featuresTrain.shape[0], -1)

    featuresVal = portionOfInitialModel.predict(testX)
    featuresVal = featuresVal.reshape(featuresVal.shape[0], -1)


    model = LogisticRegression()
    model.fit(featuresTrain, trainY)

    results = model.predict(featuresVal)
    print("results are --->")
    print (metrics.accuracy_score(results, testY))
    save_accuracy_score("FE_TL_portionOfResNet152V2_LogisticRegression", results, testY)


def featureExtractionTransferLearning_variant10():

    """
    This is the variant of feature extraction
    InceptionV3(till conv2d_50)+RandomForestClassifier_500
    Implementaion for PartB (i)
    """

    """Create a New Model Using a Portion of an Original Model"""

    initialModel = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

    #cut the original network at conv2d_50
    portionOfInitialModel= tf.keras.Model(inputs=initialModel.input, outputs=initialModel.get_layer('conv2d_50').output)

    print (portionOfInitialModel.summary())
    save_model_summary("FE_TL_portionOfInceptionV3_RandomForestClassifier_500", portionOfInitialModel)

    featuresTrain = portionOfInitialModel.predict(trainX)
    featuresTrain = featuresTrain.reshape(featuresTrain.shape[0], -1)

    featuresVal = portionOfInitialModel.predict(testX)
    featuresVal = featuresVal.reshape(featuresVal.shape[0], -1)


    model = RandomForestClassifier(n_estimators=500)
    model.fit(featuresTrain, trainY)

    results = model.predict(featuresVal)
    print("accuracy: ")
    print (metrics.accuracy_score(results, testY))
    

    save_accuracy_score("FE_TL_portionOfInceptionV3_RandomForestClassifier_500", results, testY)




def fineTuning_Variant1():

    """
    This is the variant of fine tuning
    Pretrained network VGG16 is used in Phase A by removing fully convoluted layer
    In Phase B, block4_conv1 onwards the layers are unfrozen
    Implementaion for PartB (ii)
    """

    vggModel = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

    vggModel.trainable = False
    model = tf.keras.models.Sequential()
    model.add(vggModel)
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(256, activation='relu'))  
    model.add(tf.keras.layers.Dense(17, activation='softmax'))

    print ("\n Phase A - Training Fully Connected Layers\n")
    
    print("Compiling model...")
    opt = keras.optimizers.SGD(lr=0.01)
    model.compile(loss="sparse_categorical_crossentropy",optimizer=opt,metrics=["accuracy"])
    
    #stop training when val_loss will not improve
    overfitCallback = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
    history=model.fit(trainX, trainY, epochs=NUM_EPOCHS, callbacks=[overfitCallback], batch_size=32, validation_data=(testX, testY))

    plotAccLoss(history, len(history.history['val_loss']))

    print ("\n Phase B  - Fine Tune Fully Connected Layer and Selected Convolutional Layers \n")
    vggModel.trainable = True
    trainableFlag = False
    
    for layer in vggModel.layers:
        if layer.name == 'block4_conv1':
            trainableFlag = True
        layer.trainable = trainableFlag
    vggModel.summary()

    for layer in vggModel.layers[:-3]:
        layer.trainable=False

    for layer in vggModel.layers:
        sp= '  '[len(layer.name):]
        print("sp--->",layer.name,sp,layer.trainable)
    #print("model summary--->", model.summary())

    model.compile(loss="sparse_categorical_crossentropy",optimizer=keras.optimizers.SGD(lr=1e-5),metrics=["accuracy"])
    history =model.fit(trainX, trainY, epochs=NUM_EPOCHS, batch_size=32, validation_data=(testX, testY))

    plotAccLoss(history, NUM_EPOCHS)

def fineTuning_Variant2():
    """
    Implementaion for PartB (ii)
    Details of this variant  
    Phase A: portion of InceptionV3 (till conv2d_39)
    Phase B: unfreeze the convolutional layers block with a very low learning rate
    """
    initialModel = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
    
    #cut the original network at conv2d_39
    portionOfInitialModel= tf.keras.Model(inputs=initialModel.input, outputs=initialModel.get_layer('conv2d_39').output)


    portionOfInitialModel.trainable = False
    model = tf.keras.models.Sequential()
    model.add(portionOfInitialModel)
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(256, activation='relu'))  
    model.add(tf.keras.layers.Dense(17, activation='softmax'))

    print ("\n Phase A - Training Fully Connected Layers\n")
    
    print("Compiling model...")
    opt = keras.optimizers.SGD(lr=0.01)
    model.compile(loss="sparse_categorical_crossentropy",optimizer=opt,metrics=["accuracy"])
    
    #stop training when val_loss will not improve
    overfitCallback = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
    history=model.fit(trainX, trainY, epochs=NUM_EPOCHS, callbacks=[overfitCallback], batch_size=32, validation_data=(testX, testY))

    plotAccLoss(history, len(history.history['val_loss']))

    print ("\n Phase B  - Fine Tune Fully Connected Layer and Selected Convolutional Layers \n")
    portionOfInitialModel.trainable = True
    trainableFlag = False
    
    for layer in portionOfInitialModel.layers:

        #unfreeze all the layers from block4_conv1 onwards
        if layer.name == 'conv2d_30':
            trainableFlag = True
        layer.trainable = trainableFlag
    portionOfInitialModel.summary()

    for layer in portionOfInitialModel.layers[:-3]:
        layer.trainable=False

    for layer in portionOfInitialModel.layers:
        sp= '  '[len(layer.name):]
        print("sp--->",layer.name,sp,layer.trainable)

    model.compile(loss="sparse_categorical_crossentropy",optimizer=keras.optimizers.SGD(lr=1e-5),metrics=["accuracy"])
    history =model.fit(trainX, trainY, epochs=NUM_EPOCHS, batch_size=32, validation_data=(testX, testY))

    plotAccLoss(history, NUM_EPOCHS)



def fineTuning_Variant3():

    """
    Implementaion for PartB (ii)
    This is the variant of fine tuning
    Pretrained network VGG16 is used in Phase A by removing fully convoluted layer
    In Phase B, block4_conv1 onwards the layers are unfrozen
    """

    vggModel = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

    vggModel.trainable = False
    model = tf.keras.models.Sequential()
    model.add(vggModel)
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(500, activation='relu')) 
    model.add(tf.keras.layers.Dense(17, activation='softmax'))

    print ("\n Phase A - Training Fully Connected Layers\n")
    
    print("Compiling model...")
    opt = keras.optimizers.SGD(lr=0.01)
    model.compile(loss="sparse_categorical_crossentropy",optimizer=opt,metrics=["accuracy"])
    
    #stop training when val_loss does not improve
    overfitCallback = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
    history=model.fit(trainX, trainY, epochs=NUM_EPOCHS, callbacks=[overfitCallback], batch_size=32, validation_data=(testX, testY))

    plotAccLoss(history, len(history.history['val_loss']))

    print ("\n Phase B  - Fine Tune Fully Connected Layer and Selected Convolutional Layers \n")
    vggModel.trainable = True
    trainableFlag = False
    
    for layer in vggModel.layers:
        if layer.name == 'block4_conv1':
            trainableFlag = True
        layer.trainable = trainableFlag
    vggModel.summary()

    for layer in vggModel.layers[:-3]:
        layer.trainable=False

    for layer in vggModel.layers:
        sp= '  '[len(layer.name):]
        print("sp--->",layer.name,sp,layer.trainable)
    #print("model summary--->", model.summary())

    model.compile(loss="sparse_categorical_crossentropy",optimizer=keras.optimizers.SGD(lr=1e-5),metrics=["accuracy"])
    history =model.fit(trainX, trainY, epochs=NUM_EPOCHS, batch_size=32, validation_data=(testX, testY))

    plotAccLoss(history, NUM_EPOCHS)


def fineTuning_Variant4():

    """
    Implementaion for PartB (ii)
    This is the variant of fine tuning
    Pretrained network VGG16 is used in Phase A by removing fully convoluted layer
    In Phase B, block4_conv1 onwards the layers are unfrozen
    In Phase A, no dropout are applied in the newly created fully connected layer.
    """

    vggModel = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
    vggModel.trainable = False
    model = tf.keras.models.Sequential()
    model.add(vggModel)
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(500, activation='relu')) 
    model.add(tf.keras.layers.Dense(17, activation='softmax'))
    
    print ("\n Phase A - Training Fully Connected Layers\n")
    print("Compiling model...")
    opt = keras.optimizers.SGD(lr=0.01)
    model.compile(loss="sparse_categorical_crossentropy",optimizer=opt,metrics=["accuracy"])
    #stop training when val_loss does not improve
    overfitCallback = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
    history=model.fit(trainX, trainY, epochs=NUM_EPOCHS, callbacks=[overfitCallback], batch_size=32, validation_data=(testX, testY))
    plotAccLoss(history, len(history.history['val_loss']))
    print ("\n Phase B  - Fine Tune Fully Connected Layer and Selected Convolutional Layers \n")
    vggModel.trainable = True
    trainableFlag = False
    for layer in vggModel.layers:
        if layer.name == 'block4_conv1':
            trainableFlag = True
        layer.trainable = trainableFlag
    vggModel.summary()
    for layer in vggModel.layers[:-3]:
        layer.trainable=False
    for layer in vggModel.layers:
        sp= '  '[len(layer.name):]
        print("sp--->",layer.name,sp,layer.trainable)
    #print("model summary--->", model.summary())
    model.compile(loss="sparse_categorical_crossentropy",optimizer=keras.optimizers.SGD(lr=1e-5),metrics=["accuracy"])
    history =model.fit(trainX, trainY, epochs=NUM_EPOCHS, batch_size=32, validation_data=(testX, testY))
    plotAccLoss(history, NUM_EPOCHS)

#featureExtractionTransferLearning_NN()


###############################################################################
#                        execution details
###############################################################################

"""
please create folders 'model_summary' and 'accuracy_results' as the results will be stored there
"""

"""please uncomment any of the fucntion to execute"""

#featureExtractionTransferLearning_variant1()
#featureExtractionTransferLearning_variant2()
#featureExtractionTransferLearning_variant3()
#featureExtractionTransferLearning_variant4()
#featureExtractionTransferLearning_variant5()
#featureExtractionTransferLearning_variant6()
#featureExtractionTransferLearning_variant7()
#featureExtractionTransferLearning_variant8()
#featureExtractionTransferLearning_variant9()
#featureExtractionTransferLearning_variant10()
#featureExtractionTransferLearning_variant11()

#fineTuning_Variant1()
#fineTuning_Variant2()
#fineTuning_Variant3()
#fineTuning_Variant4()

Using TensorFlow backend.


(1020, 128, 128, 3) (1020,)
(340, 128, 128, 3) (340,)
58892288/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
________

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
